In [7]:
import rasterio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import contextily as ctx
import geopandas as gpd
from shapely.geometry import mapping, box
import rasterio
from rasterstats import zonal_stats
import tempfile
from docx import Document
from docx.shared import Inches, Pt
from docx.enum.table import WD_TABLE_ALIGNMENT
from docx.enum.text import WD_ALIGN_PARAGRAPH
from docx2pdf import convert
from mpl_toolkits.axes_grid1.inset_locator import inset_axes, mark_inset
import math
from matplotlib_scalebar.scalebar import ScaleBar
import matplotlib.font_manager as fm
import os, re, tempfile, shutil, unicodedata
import pythoncom
import win32com.client as win32
from win32com.client import Dispatch
from matplotlib.patches import FancyArrow, Polygon
import metodos_mixtos as mmc
from metodos_mixtos import mapas
from metodos_mixtos import mapas_raster

In [8]:
# Cálculo de deforestación anual por polígono en formato tabla (se calcula en ha)
def def_anual(gdf, raster_path, year_min=2000, year_max=2024):
    """
    Calcula la deforestación anual en hectáreas (ha) por polígono
    usando Hansen multibanda y recorte previo.
    """
    # Códigos Hansen
    start_code = year_min - 2000
    end_code = year_max - 2000

    # Archivos temporales
    temp_dir = tempfile.mkdtemp()
    temp_shp = os.path.join(temp_dir, "temp_shape.shp")
    temp_tif = os.path.join(temp_dir, "temp_raster_clip.tif")
    gdf.to_file(temp_shp)

    # Recorte del raster
    mapas_raster.raster_clipping(temp_shp, raster_path, temp_tif)

    # Leer y filtrar raster
    with rasterio.open(temp_tif) as src:
        treecover2000 = src.read(1)
        loss = src.read(2)
        lossyear = src.read(3)

        # Máscara de pérdida válida
        mask_loss = (
            (treecover2000 > 0) &
            (loss == 1) &
            (lossyear >= start_code) &
            (lossyear <= end_code)
        )

        lossyear_masked = np.where(mask_loss, lossyear, 0)

        # Guardar el profile aquí dentro del with
        profile = src.profile

    # Guardar raster filtrado
    profile.update(dtype=rasterio.uint8, count=1)
    temp_filtered = os.path.join(temp_dir, "filtered_lossyear.tif")
    with rasterio.open(temp_filtered, "w", **profile) as dst:
        dst.write(lossyear_masked.astype(rasterio.uint8), 1)

    # Calcular estadísticas zonales por año
    stats = zonal_stats(
        gdf, temp_filtered,
        stats=['count'],
        categorical=True,
        nodata=0
    )

    results = []
    for i, stat in enumerate(stats):
        for year_code, pixel_count in stat.items():
            if isinstance(year_code, int) and year_code > 0:
                year = 2000 + year_code
                area_ha = pixel_count * (30 * 30) / 10000
                results.append({
                    'id': gdf.iloc[i].get('id', i),
                    'year': year,
                    'deforestation_ha': area_ha
                })

    df_result = pd.DataFrame(results)

    # Mensaje si no hay deforestación detectada
    if df_result.empty or df_result["deforestation_ha"].sum() == 0:
        print("⚠ No se detectó deforestación en el rango de años especificado.")

    return df_result

In [10]:
# Funciones para flecha norte y barra de escala


# Flecha del norte
def add_north_arrow(ax, xy=(0.1, 0.88), size=0.1, text="N",
                    color="black", lw=1.8):
    """
    Flecha del norte en la esquina superior izquierda (por defecto).
    - xy: base de la flecha (en fracción del eje)
    - size: largo de la flecha
    - text: etiqueta (default 'N')
    - color: color del texto y la flecha
    - lw: grosor de línea
    """
    # Texto "N" arriba de la flecha
    ax.text(xy[0], xy[1] + size + 0.02, text,
            transform=ax.transAxes, ha="center", va="bottom",
            fontsize=12, fontweight="bold", color=color, zorder=1000)

    # Flecha
    arr = FancyArrow(xy[0], xy[1], 0, size,
                     width=size*0.25, head_width=size*0.45, head_length=size*0.35,
                     length_includes_head=True, transform=ax.transAxes,
                     color=color, linewidth=lw, zorder=999)
    ax.add_patch(arr)


# Escala
def _pick_nice_length_km(width_km):
    """Elige un largo 'bonito' ~1/5 del ancho del AOI."""
    target = max(1, width_km/5)
    for nice in [1,2,5,10,20,25,50,100,200,250,500,1000]:
        if target <= nice:
            return nice
    return 1000

def _loc_xy0(ax, loc, pad_frac_x, pad_frac_y):
    """Devuelve esquina (x0,y0) según loc, en unidades de datos del eje."""
    X0, X1 = ax.get_xlim(); Y0, Y1 = ax.get_ylim()
    xpad = (X1 - X0) * pad_frac_x
    ypad = (Y1 - Y0) * pad_frac_y
    if loc == "lower right":
        return (X1 - xpad, Y0 + ypad)
    elif loc == "lower left":
        return (X0 + xpad, Y0 + ypad)
    elif loc == "upper right":
        return (X1 - xpad, Y1 - ypad)
    elif loc == "upper left":
        return (X0 + xpad, Y1 - ypad)
    else:
        return (X1 - xpad, Y0 + ypad)  # por defecto lower right

# Escala en lon/lat (EPSG:4326)
def add_scalebar_lonlat(ax, gdf_wgs, loc="lower right",
                        pad_frac=(0.06, 0.06), height_frac=0.012,
                        font=9, segments=2):
    """
    Barra de escala para mapas en WGS84 (grados). Aproxima metros/° por latitud.
    - loc: 'lower/upper left/right'
    - pad_frac: (x,y) margen como fracción del rango del eje
    - height_frac: alto de la barra como fracción del rango vertical
    - segments: bloques alternados (2 típico)
    """
    # Asegura 4326
    if gdf_wgs.crs is None:
        gdf_wgs = gdf_wgs.set_crs(4326)
    else:
        gdf_wgs = gdf_wgs.to_crs(4326)

    # Ancho del AOI en km (vía 3857 para tener métricas correctas)
    bounds_m = gdf_wgs.to_crs(3857).total_bounds
    width_km = (bounds_m[2] - bounds_m[0]) / 1000.0
    length_km = _pick_nice_length_km(width_km)

    # Metros por grado de longitud a la latitud central (por crs)
    cy = float(gdf_wgs.geometry.unary_union.centroid.y)
    meters_per_deg_lon = 111_320 * math.cos(math.radians(cy))
    deg_len = max(1e-9, (length_km * 1000) / max(1e-9, meters_per_deg_lon))

    # Posición base
    X0, X1 = ax.get_xlim(); Y0, Y1 = ax.get_ylim()
    x0_base, y0_base = _loc_xy0(ax, loc, pad_frac[0], pad_frac[1])

    # Ajuste según esquina (x0,y0)
    if "right" in loc:
        x0 = x0_base - deg_len
    else:
        x0 = x0_base
    if "upper" in loc:
        
        y0 = y0_base - (Y1 - Y0) * height_frac
    else:
        y0 = y0_base

    # Alto relativo
    height = (Y1 - Y0) * height_frac

    # Dibujar segmentos alternados
    seg_len = deg_len / max(1, segments)
    for i in range(segments):
        rect = mpatches.Rectangle(
            (x0 + i*seg_len, y0), seg_len, height,
            facecolor=("black" if i % 2 == 0 else "white"),
            edgecolor="black", linewidth=0.8, zorder=6
        )
        ax.add_patch(rect)

    # Etiqueta
    ax.text(x0 + deg_len/2, y0 + height*1.35, f"{int(length_km)} km",
            ha="center", va="bottom", fontsize=font, color="black",
            bbox=dict(facecolor="white", edgecolor="0.8", alpha=0.85, pad=1.2),
            zorder=7)

# Escala en metros (crs 3857)
def add_scalebar_meter(ax, gdf_m, loc="lower right",
                       pad_frac=(0.05, 0.05), height_frac=0.012,
                       font=10, segments=2):
    """
    Barra de escala para CRS en metros (UTM/3857).
    - height_frac: alto relativo al rango Y del eje (robusto a cambios de zoom)
    """
    # Se asume crs en metros
    xmin, ymin, xmax, ymax = gdf_m.total_bounds
    width_km = (xmax - xmin)/1000.0
    length_km = _pick_nice_length_km(width_km)
    length_m  = length_km * 1000.0

    # Posición base
    X0, X1 = ax.get_xlim(); Y0, Y1 = ax.get_ylim()
    x0_base, y0_base = _loc_xy0(ax, loc, pad_frac[0], pad_frac[1])

    
    if "right" in loc:
        x0 = x0_base - length_m
    else:
        x0 = x0_base
    if "upper" in loc:
        y0 = y0_base - (Y1 - Y0) * height_frac
    else:
        y0 = y0_base

    # Alto relativo
    height = (Y1 - Y0) * height_frac

    # Segmentos
    seg_len = length_m / max(1, segments)
    for i in range(segments):
        rect = mpatches.Rectangle(
            (x0 + i*seg_len, y0), seg_len, height,
            facecolor=("black" if i % 2 == 0 else "white"),
            edgecolor="black", linewidth=0.8, zorder=6
        )
        ax.add_patch(rect)

    # Etiqueta
    label = f"{int(length_km)} km" if length_km >= 1 else f"{int(length_m)} m"
    ax.text(x0 + length_m/2, y0 + height*1.35, label,
            ha="center", va="bottom", fontsize=font, color="black",
            bbox=dict(facecolor="white", edgecolor="0.8", alpha=0.85, pad=1.2),
            zorder=7)

# Fuente del basemap
def add_attribution(ax, text, fontsize=6, alpha=0.6, loc="lower left", pad=0.01):
    """
    Texto pequeño para la fuente del basemap en fracción del eje.
    """
    x = pad if "left" in loc else 1 - pad
    y = pad if "lower" in loc else 1 - pad
    ax.text(x, y, text, transform=ax.transAxes,
            ha=("left" if "left" in loc else "right"),
            va=("bottom" if "lower" in loc else "top"),
            fontsize=fontsize, color="0.2",
            bbox=dict(facecolor="white", alpha=alpha, edgecolor="none",
                      boxstyle="round,pad=0.15"),
            zorder=2000)



In [11]:
# Mapa de deforestación por lote

def plot_deforestation_map(
    raster_path, gdf, names_column, name_of_area,
    year_start, year_end, output_folder=".", basemap=False
):
    safe_name = str(name_of_area).replace(" ", "_").replace("/", "_")
    output_path = os.path.join(output_folder, f"deforestacion_{safe_name}_{year_start}_a_{year_end}.png")

    temp_dir = tempfile.mkdtemp()
    temp_shp = os.path.join(temp_dir, "temp_shape.shp")
    temp_tif = os.path.join(temp_dir, "temp_raster_clip.tif")
    gdf.to_file(temp_shp)
    mapas_raster.raster_clipping(temp_shp, raster_path, temp_tif)

    with rasterio.open(temp_tif) as src:
        raster_crs = src.crs
        if gdf.crs != raster_crs: gdf = gdf.to_crs(raster_crs)

        band_count = src.count
        start_code = year_start - 2000
        end_code = year_end - 2000

        if band_count == 1:
            lossyear = src.read(1)
            loss_mask = ((lossyear >= start_code) & (lossyear <= end_code)).astype(np.uint8) * 255
            preserved_mask = (lossyear == 0).astype(np.uint8) * 255
        elif band_count >= 3:
            treecover2000 = src.read(1); loss = src.read(2); lossyear = src.read(3)
            valid_forest = treecover2000 > 0
            loss_mask = (valid_forest & (loss == 1) & (lossyear >= start_code) & (lossyear <= end_code)).astype(np.uint8) * 255
            preserved_mask = (valid_forest & ((loss == 0) | (lossyear > end_code))).astype(np.uint8) * 255
        else:
            raise ValueError("El raster debe tener 1 banda o 3 bandas Hansen.")

        extent = [src.bounds.left, src.bounds.right, src.bounds.bottom, src.bounds.top]

    # Plot 
    fig, ax = plt.subplots(figsize=(10, 8), constrained_layout=True)

    ax.imshow(preserved_mask, cmap='Greens', extent=extent)
    ax.imshow(loss_mask, cmap='Reds',   extent=extent, alpha=0.6)

    gdf.boundary.plot(ax=ax, color='black', linewidth=1)
    for _, row in gdf.iterrows():
        c = row.geometry.centroid
        ax.annotate(text=str(row.get(names_column, "")), xy=(c.x, c.y), ha='center', fontsize=5, color='black')

    legend1 = mpatches.Patch(color='green', label=f'Bosque en {year_end}')
    legend2 = mpatches.Patch(color='red',   label=f'Pérdida {year_start}–{year_end}')
    ax.legend(handles=[legend1, legend2], loc='upper right', frameon=True)

    ax.set_title(f'Pérdida de bosque entre {year_start} y {year_end} en {name_of_area}')
    ax.set_xticks([]); ax.set_yticks([])

    DEF_BUFFER = 0.35  
    xmin, ymin, xmax, ymax = gdf.total_bounds
    dx = (xmax - xmin) * DEF_BUFFER; dy = (ymax - ymin) * DEF_BUFFER
    ax.set_xlim(xmin - dx, xmax + dx)
    ax.set_ylim(ymin - dy, ymax + dy)

   
    add_north_arrow(ax, xy=(0.07, 0.85), size=0.05, color="black")


    # Escala abajo-derecha en crs 4326. Si tu raster está en 3857, usa add_scalebar_meter.
    gdf_wgs = gdf.to_crs(4326) if (gdf.crs and gdf.crs.to_epsg() != 4326) else gdf
    add_scalebar_lonlat(ax, gdf_wgs=gdf_wgs, loc="lower right", segments=2)
    add_attribution(ax, "Fuente: Hansen Global Forest Change 2024",
                fontsize=10, loc="lower left")


    plt.savefig(output_path, bbox_inches='tight', dpi=400)
    plt.close()
    print(f"Mapa guardado en: {output_path}")
    return output_path

In [23]:
# ================== Imports ==================
import os, json, base64
from pathlib import Path
import geopandas as gpd
import pandas as pd
from shapely.geometry import mapping

# ================== CONFIG ===================
# Input data
shp_path    = r"C:\Users\laura\OneDrive - Vestigium Métodos Mixtos Aplicados SAS\Archivos de Daniel Wiesner - simbyp_data\deforestation_reports\Shapes PSA\areas_priorizadas_psah.shp"
hansen_path = r"C:\Users\laura\OneDrive - Vestigium Métodos Mixtos Aplicados SAS\Archivos de Daniel Wiesner - simbyp_data\deforestation_reports\Hansen Colombia 2024\hansen_treecover_SDP_2024.tif"

# Output folder
out_dir     = Path(r"C:\Users\laura\OneDrive\Documents\GitHub\Bosques_Bogotá_Nuevo\bosques-bog\deforestation_reports\output\reportes_html")
out_dir.mkdir(parents=True, exist_ok=True)

# Field names in the shapefile
objectid_field = "OBJECTID"
lot_field      = "lotCodigo"
name_field     = "PreDirecc"   # parcel display name
area_field     = "Area_ha"     # added to Resumen ("Área en hectáreas")

# Build years (used to compute totals, build the PNG map, and in filenames)
YEAR_MIN_BUILD   = 2000
YEAR_MAX_BUILD   = 2024

# Allowed range for user selection in Explorer + report clamping
YEAR_MIN_ALLOWED = 2000
YEAR_MAX_ALLOWED = 2024

# Optional logo in the header (embedded base64). Leave "" to hide.
LOGO_PATH = r"C:\Users\laura\OneDrive\Documents\GitHub\Bosques_Bogotá_Nuevo\bosques-bog\deforestation_reports\data\Logo_SDP.jpeg"
LOGO_HEIGHT_PX = 36

# ================ Helpers ====================
def _to_fc(geom):
    return {
        "type":"FeatureCollection",
        "features":[{"type":"Feature","properties":{},"geometry":mapping(geom)}]
    }

def _b64(p: Path) -> str:
    return base64.b64encode(p.read_bytes()).decode("ascii") if p and p.exists() else ""

def _safe(s: str) -> str:
    # safe for filenames
    return "".join(ch if ch.isalnum() or ch in ("-", "_") else "_" for ch in str(s))

# =============== Load parcels =================
gdf_all = gpd.read_file(shp_path)
if gdf_all.crs is None:
    raise ValueError("El shapefile no tiene CRS. Asigna uno (p.ej., EPSG:4326) antes de continuar.")
gdf_all = gdf_all.to_crs(4326)

# Ensure key columns are strings and create unique pair key (no geometry merges)
gdf_all[objectid_field] = gdf_all[objectid_field].astype(str)
gdf_all[lot_field]      = gdf_all[lot_field].astype(str)
gdf_all["_pair_key"]    = gdf_all[objectid_field] + "__" + gdf_all[lot_field]

# Keep first row per (OBJECTID, lotCodigo) pair (no dissolve)
gdf_pairs = gdf_all.drop_duplicates(subset=["_pair_key"]).copy()

# Pre-embed logo (optional)
logo_b64 = _b64(Path(LOGO_PATH))
logo_img = f'<img src="data:image/jpeg;base64,{logo_b64}" alt="Logo" style="height:{LOGO_HEIGHT_PX}px"/>' if logo_b64 else ""

manifest = []

# ========= Generate one report per (OBJECTID, lotCodigo) =========
for _, row in gdf_pairs.iterrows():
    objid = row[objectid_field]
    lot   = row[lot_field]
    name  = str(row.get(name_field, "sin_nombre"))
    area_ha_val = row.get(area_field, None)
    try:
        area_ha_fmt = f"{float(area_ha_val):,.2f}".replace(",", "X").replace(".", ",").replace("X", ".") if area_ha_val is not None else "—"
    except Exception:
        area_ha_fmt = "—"

    # One-row GeoDataFrame (no merge)
    parcel_gdf = gpd.GeoDataFrame([row], crs=gdf_all.crs)

    # 1) Data table for build range (2000–2024) using YOUR function
    df_def = def_anual(parcel_gdf, hansen_path, YEAR_MIN_BUILD, YEAR_MAX_BUILD)
    total_loss = float(df_def["deforestation_ha"].sum()) if not df_def.empty else 0.0

    # If no rows or zero sum, show your message (no table)
    if df_def.empty or total_loss == 0:
        table_block_html = "<div class='text-warning'>⚠ No se detectó deforestación en el rango de años especificado.</div>"
    else:
        table_block_html = (
            df_def.rename(columns={"year": "Año", "deforestation_ha": "Ha deforestadas"})
                 .sort_values("Año")
                 .to_html(index=False, classes=["table","table-striped","table-sm","mb-0"], border=0)
        )

    # 2) Context polygon GeoJSON
    geojson_str = json.dumps(_to_fc(row.geometry), ensure_ascii=False)

    # 3) Deforestation map image (always) using YOUR function
    safe_obj = _safe(objid)
    safe_lot = _safe(lot)
    defmap_png = out_dir / f"defmap_{safe_obj}_{safe_lot}_{YEAR_MIN_BUILD}_{YEAR_MAX_BUILD}.png"
    try:
        plot_deforestation_map(
            aoi_gdf=gdf_all,               # or AOI gdf if you use a different one
            poly_gdf=parcel_gdf,
            output_path=str(defmap_png),
            year_min=YEAR_MIN_BUILD, year_max=YEAR_MAX_BUILD
        )
    except Exception as e:
        print(f"⚠ No se pudo generar el mapa de deforestación para (OBJECTID={objid}, lotCodigo={lot}): {e}")
    img_src = f"data:image/png;base64,{_b64(defmap_png)}" if defmap_png.exists() else ""

    # 4) Report filename (includes build years)
    report_file = out_dir / f"report_{safe_obj}_{safe_lot}_{YEAR_MIN_BUILD}_{YEAR_MAX_BUILD}.html"

    # 5) Title line (your requested format)
    title_line = f"{name} (OBJECT ID: {objid} - lotCodigo: {lot})"

    # 6) HTML template (Map first, then Table; logo; back button; **Esri World Imagery + Hillshade**)
    report_html = f"""
<!doctype html>
<html lang="es">
<head>
  <meta charset="utf-8"/>
  <meta name="viewport" content="width=device-width, initial-scale=1"/>
  <title>Reporte de alertas – {name} (OBJECT ID: {objid} - lotCodigo: {lot})</title>
  <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.2/dist/css/bootstrap.min.css"/>
  <link rel="stylesheet" href="https://unpkg.com/leaflet@1.9.4/dist/leaflet.css"/>
  <style>
    :root {{
      --red-bogota:#c8102e;
      --bg:#ffffff; --text:#111; --muted:#555; --card:#f7f7f8; --border:#e5e7eb;
    }}
    body {{ background:var(--bg); color:var(--text); }}
    .hero {{ background:var(--red-bogota); color:#fff; padding:0.9rem 0; }}
    .hero .brand {{ display:flex; gap:12px; align-items:center; }}
    .muted {{ color:var(--muted); }}
    .card-soft {{ background:var(--card); border:1px solid var(--border); border-radius:12px; }}
    .section-title {{ font-weight:700; font-size:1.15rem; margin-bottom:.5rem; }}
    #map {{ height:420px; border-radius:12px; }}
    img.map-static {{ width:100%; max-height:520px; object-fit:contain; border-radius:12px; border:1px solid var(--border); }}
    .label-kv span {{ display:inline-block; min-width:9.5rem; color:var(--muted); }}
  </style>
</head>
<body>
  <!-- Header (logo + title + back button) -->
  <div class="hero">
    <div class="container d-flex justify-content-between align-items-center">
      <div class="brand">
        {logo_img}
        <div>
          <div class="h5 m-0">Reporte de alertas de deforestación en predios del esquema PSAH</div>
          <div class="small">Secretaría Distrital de Planeación – Bogotá</div>
        </div>
      </div>
      <a href="index.html" class="btn btn-light btn-sm">← Volver al Explorador</a>
    </div>
  </div>

  <div class="container my-4">
    <!-- Title / Params -->
    <div class="mb-3">
      <div class="h4 mb-1" id="title-years">{title_line}</div>
      <div class="muted" id="years-sub">Rango: {YEAR_MIN_BUILD}–{YEAR_MAX_BUILD}</div>
    </div>

    <!-- Description -->
    <div class="card-soft p-3 mb-4">
      <div class="section-title">Metodología</div>
      <p class="mb-2">
        Este reporte presenta las alertas de deforestación del predio <strong>{name}</strong> entre
        <span id="desc-years">{YEAR_MIN_BUILD} y {YEAR_MAX_BUILD}</span>. Incluye una caracterización geográfica,
        una tabla con hectáreas de pérdida por año y un mapa de deforestación.
      </p>
      <div class="small muted">Fuente de información: Hansen – Global Forest Change.</div>
    </div>

    <!-- Context map + resumen -->
    <div class="row g-3 mb-4">
      <div class="col-12 col-lg-8">
        <div class="card-soft p-3 h-100">
          <div class="section-title">Mapa de contextualización geográfica</div>
          <div id="map"></div>
          <div class="small muted mt-2">Figura 1. Mapa de ubicación del predio {name}. Fuente: Elaboración propia.</div>
        </div>
      </div>
      <div class="col-12 col-lg-4">
        <div class="card-soft p-3 h-100">
          <div class="section-title">Resumen</div>
          <div class="label-kv"><span>Predio</span> {name}</div>
          <div class="label-kv"><span>OBJECT ID</span> {objid}</div>
          <div class="label-kv"><span>lotCodigo</span> {lot}</div>
          <div class="label-kv"><span>Área en hectáreas</span> {area_ha_fmt}</div>
          <div class="label-kv"><span>Rango</span> <span id="kv-years">{YEAR_MIN_BUILD}–{YEAR_MAX_BUILD}</span></div>
          <div class="label-kv"><span>Pérdida total</span> <strong id="kv-total">{total_loss:.2f} ha</strong></div>
        </div>
      </div>
    </div>

    <!-- Deforestation map FIRST -->
    <div class="card-soft p-3 mb-4">
      <div class="section-title">Mapa de deforestación</div>
      {f'<img class="map-static" src="{img_src}" alt="Mapa de deforestación"/>' if img_src else "<div class='text-warning'>⚠ No se pudo renderizar el mapa de deforestación para este predio.</div>"}
      <div class="small muted mt-2">Figura 2. Mapa de deforestación en el predio {name}. Fuente: Elaboración propia con base en Hansen.</div>
    </div>

    <!-- Table / or message when no data -->
    <div class="card-soft p-3 mb-5">
      <div class="section-title">Hectáreas de pérdida de cobertura arbórea por año</div>
      <div id="table-container">
        {table_block_html}
      </div>
      <div class="small muted mt-2">Tabla 1. Hectáreas de pérdida por año. Fuente: Elaboración propia con base en Hansen.</div>
    </div>
  </div>

  <script src="https://unpkg.com/leaflet@1.9.4/dist/leaflet.js"></script>
  <script>
    // Embedded data
    const parcel = {geojson_str};
    const fullTableHTML = `{table_block_html.replace('`','\\u0060')}`;

    // URL params (?start=YYYY&end=YYYY) — default to build range, clamped to 2000–2024
    const p = new URLSearchParams(window.location.search);
    const start = Math.max({YEAR_MIN_ALLOWED}, parseInt(p.get('start') || '{YEAR_MIN_BUILD}', 10));
    const end   = Math.min({YEAR_MAX_ALLOWED}, parseInt(p.get('end')   || '{YEAR_MAX_BUILD}', 10));

    // Update headings with chosen years
    document.getElementById('years-sub').textContent  = `Rango: ${{start}}–${{end}}`;
    document.getElementById('desc-years').textContent = `${{start}} y ${{end}}`;
    document.getElementById('kv-years').textContent   = `${{start}}–${{end}}`;

    // Map (Esri World Imagery + Hillshade) — very detailed, good for small parcels
    const map = L.map('map', {{ zoomSnap: 0.25, zoomDelta: 0.25 }});
    const imagery = L.tileLayer(
      'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{{z}}/{{y}}/{{x}}',
      {{ maxZoom: 20, maxNativeZoom: 19, detectRetina: true, attribution: 'Imagery © Esri & sources' }}
    ).addTo(map);
    const hillshade = L.tileLayer(
      'https://server.arcgisonline.com/ArcGIS/rest/services/Elevation/World_Hillshade/MapServer/tile/{{z}}/{{y}}/{{x}}',
      {{ maxZoom: 20, opacity: 0.35, attribution: 'Hillshade © Esri' }}
    ).addTo(map);

    const layer = L.geoJSON(parcel, {{ style: {{ color:'#c8102e', weight:2, fillOpacity:0.15 }} }}).addTo(map);
    map.fitBounds(layer.getBounds(), {{ padding: [20,20] }});
    L.control.scale({{ metric:true, imperial:false }}).addTo(map);

    // Table filter by chosen years & recompute total (only if we actually have a table)
    (function filterTable(){{
      const container = document.getElementById('table-container');
      const tb = container.querySelector('table');
      if (!tb) return; // it's the 'no data' message
      const rows = Array.from(tb.querySelectorAll('tbody tr'));
      let total = 0;
      rows.forEach(tr => {{
        const yr = parseInt(tr.children[0].textContent.trim(), 10);
        const ha = parseFloat(tr.children[1].textContent.trim()) || 0;
        const show = (yr >= start && yr <= end);
        tr.style.display = show ? '' : 'none';
        if (show) total += ha;
      }});
      document.getElementById('kv-total').textContent = `${{total.toFixed(2)}} ha`;
    }})();
  </script>
</body>
</html>
"""
    report_html = report_html.replace("\r\n", "\n")
    report_file.write_text(report_html, encoding="utf-8")

    manifest.append({
        "pair": f"{objid} — {lot}",
        "title": f"{name} (OBJECT ID: {objid} - lotCodigo: {lot})",
        "file": report_file.name
    })

# ================= Explorer (index.html) =================
# Logo also in first HTML
index_logo_img = logo_img

options_html = "\n".join(
    [f"<option value='{m['file']}'>{m['title']}</option>"
     for m in sorted(manifest, key=lambda d: d["title"].lower())]
)

index_html = f"""
<!doctype html>
<html lang="es"><head>
  <meta charset="utf-8"/>
  <meta name="viewport" content="width=device-width, initial-scale=1"/>
  <title>Explorador de reportes</title>
  <link rel="stylesheet" href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.2/dist/css/bootstrap.min.css"/>
  <style>
    .hero {{ background:#c8102e; color:#fff; padding:0.9rem 0; }}
    .brand {{ display:flex; gap:12px; align-items:center; }}
    .card-soft {{ background:#f7f7f8; border:1px solid #e5e7eb; border-radius:12px; }}
  </style>
</head><body>
  <div class="hero">
    <div class="container d-flex justify-content-between align-items-center">
      <div class="brand">
        {index_logo_img}
        <div class="h5 m-0">Explorador de reportes – PSAH</div>
      </div>
    </div>
  </div>

  <div class="container my-4">
    <div class="card-soft p-3 mb-3">
      <div class="row g-3 align-items-end">
        <div class="col-12">
          <label class="form-label">Predio (OBJECTID — lotCodigo)</label>
          <select id="fileSel" class="form-select">{options_html}</select>
        </div>
        <div class="col-6 col-md-3">
          <label class="form-label">Año inicio</label>
          <input id="start" type="number" min="{YEAR_MIN_ALLOWED}" max="{YEAR_MAX_ALLOWED}"
                 value="{YEAR_MIN_BUILD}" class="form-control"/>
        </div>
        <div class="col-6 col-md-3">
          <label class="form-label">Año fin</label>
          <input id="end" type="number" min="{YEAR_MIN_ALLOWED}" max="{YEAR_MAX_ALLOWED}"
                 value="{YEAR_MAX_BUILD}" class="form-control"/>
        </div>
        <div class="col-12">
          <button id="go" class="btn btn-primary">Abrir reporte</button>
        </div>
      </div>
    </div>

    <div class="card-soft p-3">
      <div class="fw-bold mb-2">Reportes generados</div>
      <ul class="mb-0">
        {"".join([f"<li><a href='{m['file']}'>{m['title']}</a></li>" for m in sorted(manifest, key=lambda d: d['title'].lower())])}
      </ul>
    </div>
  </div>

<script>
document.getElementById('go').onclick = function() {{
  const f = document.getElementById('fileSel').value;  // exact filename
  const start = Math.max({YEAR_MIN_ALLOWED}, parseInt(document.getElementById('start').value, 10));
  const end   = Math.min({YEAR_MAX_ALLOWED}, parseInt(document.getElementById('end').value, 10));
  // Open the selected report with chosen range as URL params
  window.location.href = `${{f}}?start=${{start}}&end=${{end}}`;
}};
</script>
</body></html>
"""
(out_dir/"index.html").write_text(index_html, encoding="utf-8")

print(f"✓ Generated {len(manifest)} reports (unique OBJECTID–lotCodigo) + index.html in {out_dir.resolve()}")


⚠ No se detectó deforestación en el rango de años especificado.
⚠ No se pudo generar el mapa de deforestación para (OBJECTID=9.0, lotCodigo=102909000022): plot_deforestation_map() got an unexpected keyword argument 'aoi_gdf'
⚠ No se pudo generar el mapa de deforestación para (OBJECTID=18.0, lotCodigo=102910000026): plot_deforestation_map() got an unexpected keyword argument 'aoi_gdf'
⚠ No se detectó deforestación en el rango de años especificado.
⚠ No se pudo generar el mapa de deforestación para (OBJECTID=35.0, lotCodigo=102910000026): plot_deforestation_map() got an unexpected keyword argument 'aoi_gdf'
⚠ No se detectó deforestación en el rango de años especificado.
⚠ No se pudo generar el mapa de deforestación para (OBJECTID=37.0, lotCodigo=102909000005): plot_deforestation_map() got an unexpected keyword argument 'aoi_gdf'
⚠ No se detectó deforestación en el rango de años especificado.
⚠ No se pudo generar el mapa de deforestación para (OBJECTID=40.0, lotCodigo=102909000005): plot_